In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv("./data/train_preproc1.csv")

In [41]:
def make_seq_data(train_x, train_y, test_x, test_y, pred_days, seq_len):
    train_x_seq = []
    train_y_seq = []
    test_x_seq = []
    test_y_seq = []
    
    for i in range(seq_len, len(train_x)-pred_days):
        train_x_seq.append(train_x[i - seq_len:i])
        train_y_seq.append(train_y[i + pred_days - 1:i + pred_days])

    for i in range(seq_len, len(test_x)-pred_days):
        test_x_seq.append(test_x[i - seq_len:i])
        test_y_seq.append(test_y[i + pred_days - 1:i + pred_days])

    train_x_seq, train_y_seq = np.array(train_x_seq), np.array(train_y_seq)
    test_x_seq, test_y_seq = np.array(test_x_seq), np.array(test_y_seq)

    return train_x_seq, train_y_seq, test_x_seq, test_y_seq

In [55]:
import matplotlib.pyplot as plt

class lstm_model():
    def __init__(self, input_shape, learning_rate=1e-2, load_path=None):
        
        model = Sequential()
        model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
        model.add(LSTM(64, return_sequences=True))
        model.add(LSTM(32, return_sequences=True))
        model.add(Dense(1))
        self.model = model
        self.optimizer = Adam(learning_rate=learning_rate)
        
        if load_path is not None:
            self.load_model(load_path)
    
    def save_model(self, path):
        self.model.save_weights(path)
    
    def load_model(self, path):
        self.model.load_weights(path)

    # 학습 후 loss 그래프 그림
    def train(self, train_x, train_y, epochs=100, batch_size=32, validation_split=0.1, loss='mse'):
        self.model.compile(optimizer=self.optimizer, loss=loss)
        history = self.model.fit(train_x, train_y,
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 validation_split=validation_split)
        plt.plot(history.history['loss'], label='Training loss')
        plt.plot(history.history['val_loss'], label='Validation loss')
        plt.legend()
        plt.show()

        self.history = history

    def predict(self, x):
        pred = self.model.predict(x)
        return pred
    
    def plot_pred(self, y, pred_y):
        plt.figure(figsize=(20, 10))
        plt.plot(y, label='GT')
        plt.plot(pred_y, label='prediction')
        plt.legend()
        plt.show()

    def pred_plot(self, x, y):
        pred = self.predict(x)
        self.plot_pred(y, pred)

In [56]:
model_dic = {}
pred_days = 1
seq_len = 14

epochs = 15
batch_size = 16
learning_rate = 5e-2
loss = 'mse'

for i in tqdm(train.item.unique()):
    train_now = train.loc[train.item == i]
    y = train_now.price
    x = train_now.drop(['item', 'price'], axis=1)

    n_train = int(len(x) * 0.9)

    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    
    train_x, test_x = x[:n_train], x[n_train:]
    train_y, test_y = y[:n_train], y[n_train:]

    train_x, train_y, test_x, test_y = make_seq_data(train_x, train_y, test_x, test_y, pred_days, seq_len)

    lstm = lstm_model(train_x.shape[1:3],
                      learning_rate=learning_rate)
    
    lstm.train(train_x, train_y,
               epochs=epochs,
               batch_size=batch_size,
               loss=loss)
    lstm.pred_plot(test_x, test_y)

    model_dic[i] = (lstm, scaler)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/15
771/771 [==============================] - 20s 19ms/step - loss: 15678936.0000 - val_loss: 7693878.5000
Epoch 2/15
771/771 [==============================] - 14s 18ms/step - loss: 10820703.0000 - val_loss: 6914685.5000
Epoch 3/15
771/771 [==============================] - 14s 18ms/step - loss: 8214698.0000 - val_loss: 5731632.5000
Epoch 4/15
771/771 [==============================] - 14s 18ms/step - loss: 7030718.5000 - val_loss: 6948735.5000
Epoch 5/15
771/771 [==============================] - 13s 18ms/step - loss: 6822072.5000 - val_loss: 6283723.5000
Epoch 6/15
771/771 [==============================] - 12s 15ms/step - loss: 6526756.5000 - val_loss: 6500222.0000
Epoch 7/15
771/771 [==============================] - 12s 16ms/step - loss: 6611402.0000 - val_loss: 6552783.0000
Epoch 8/15
771/771 [==============================] - 13s 17ms/step - loss: 6407863.0000 - val_loss: 6218852.0000
Epoch 9/15
771/771 [==============================] - 13s 17ms/step - loss: 6213693.00

In [54]:
lstm.predict(test_x)

48/48 [==============================] - 0s 5ms/step


array([[[2664.0383 ],
        [3483.0786 ],
        [3793.0288 ],
        ...,
        [3877.2974 ],
        [3885.6306 ],
        [4164.342  ]],

       [[2655.235  ],
        [3720.8848 ],
        [4011.5564 ],
        ...,
        [3842.2432 ],
        [3842.2432 ],
        [3842.2441 ]],

       [[2050.0713 ],
        [3085.9905 ],
        [3918.2295 ],
        ...,
        [4210.9473 ],
        [4211.6836 ],
        [4211.6895 ]],

       ...,

       [[2270.471  ],
        [2555.6663 ],
        [2720.8481 ],
        ...,
        [3108.0225 ],
        [3061.798  ],
        [2787.94   ]],

       [[2191.68   ],
        [3343.579  ],
        [3677.5403 ],
        ...,
        [4238.636  ],
        [4238.6377 ],
        [4238.6377 ]],

       [[1228.3014 ],
        [1826.8442 ],
        [1720.4707 ],
        ...,
        [ 642.89734],
        [ 432.42792],
        [ 666.9737 ]]], dtype=float32)

In [7]:
train_now.drop(['item', 'price'], axis=1)

,year,month,day,dow,is_holiday,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2019,1,1,1,1,1,0,0,0,0,0,1,0
1,2019,1,2,2,0,1,0,0,0,0,0,1,0
2,2019,1,3,3,0,1,0,0,0,0,0,1,0
3,2019,1,4,4,0,1,0,0,0,0,0,1,0
4,2019,1,5,5,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15225,2023,2,27,0,0,0,0,0,0,1,0,0,1
15226,2023,2,28,1,0,0,0,0,0,1,0,0,1
15227,2023,3,1,2,1,0,0,0,0,1,0,0,1
15228,2023,3,2,3,0,0,0,0,0,1,0,0,1
